In [36]:
import time
from os import path
from pathlib import Path
from datetime import datetime

import astropy.units as u
import numpy as np
from astropy.table import Table, vstack, unique


from astroquery.vizier import Vizier
from astroquery.ipac.ned import Ned
#from astroquery.ned.core import RemoteServiceError, TableParseError

import requests
from requests import Request, Session

from numpy import unique as uniq

from astropy.coordinates import SkyCoord
from astropy.table import Table
from matplotlib import pyplot as plt

In [12]:
path_raw='../Datos/Raw/'
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object
path_phot_cds = '../Datos/Work/Phot_Tables_CDS/'
date_sample = 'Apr_12_2022'
NeIR_obj_t = Table.read(path_raw+'Obj_Ne-IR_'+date_sample+'.vot',format='votable')
#NeIR_obj_t.show_in_notebook()

In [20]:
i = 0
NeIR_obj_t[i]

Main_Id,Id_used_NED,RA,DEC,otype_txt,redshift,rvz_type,otypes,cds_url,SMB_sample_index,Ne_IR_info,NED_Ne_IR_info
object,object,float64,float64,object,float64,str1,object,str93,float64,str14,str14
3C 234.0,3C 234.0,150.45634473807,28.7858176354,Sy1,0.18494,z,AGN|G |IR |QSO|Rad|Sy1|X |blu|gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=150.45634473807%2028.7858176354&-c.rs=5,283.0,NeII|NeIII|NeV,NeII|NeIII|NeV


In [22]:
cds_t = Table.read(path_phot_cds+NeIR_obj_t['Main_Id'][i]+'_CDS_phot_tables'+'.vot', 
                      format='votable')
ned_t = Table.read(path_phot_ned+NeIR_obj_t['Main_Id'][i]+'_NED_phot_tables'+'.vot', 
                      format='votable')

In [23]:
cds_t.show_in_notebook()

idx,_RAJ2000,_DEJ2000,_tabname,_ID,_tab1_31,_tab1_32,sed_freq,sed_flux,sed_eflux,sed_filter,Bibcode
,deg,deg,,,d,s,GHz,Jy,Jy,---,
0,150.456380600,28.785966700,I/297/out,"NOMAD1===1187-0182426&-c=150.4563806 +28.7859667,eq=J2000&-c.rs=",--,0.0,138550,0.00501,--,2MASS:Ks,2004AAS...205.4815Z
1,150.456380600,28.785966700,I/297/out,"NOMAD1===1187-0182426&-c=150.4563806 +28.7859667,eq=J2000&-c.rs=",--,0.0,181750,0.00215,--,2MASS:H,2004AAS...205.4815Z
2,150.456380600,28.785966700,I/297/out,"NOMAD1===1187-0182426&-c=150.4563806 +28.7859667,eq=J2000&-c.rs=",--,0.0,241960,0.00116,--,2MASS:J,2004AAS...205.4815Z
3,150.456445000,28.785850000,I/305/out,"GSC2.3===N9AE005113&-c=150.456445 +28.785850,eq=J2000&-c.rs=0.00",--,0.0,382540,0.0021,0.00083,POSS-II:i,2008AJ....136..735L
4,150.456445000,28.785850000,I/305/out,"GSC2.3===N9AE005113&-c=150.456445 +28.785850,eq=J2000&-c.rs=0.00",--,0.0,468430,0.000992,0.00039,POSS-II:F,2008AJ....136..735L
5,150.456445000,28.785850000,I/305/out,"GSC2.3===N9AE005113&-c=150.456445 +28.785850,eq=J2000&-c.rs=0.00",--,0.0,640580,0.000753,0.00031,POSS-II:J,2008AJ....136..735L
6,150.456445000,28.785850000,I/305/out,"GSC2.3===N9AE005113&-c=150.456445 +28.785850,eq=J2000&-c.rs=0.00",--,0.0,674900,0.000364,0.00012,Johnson:B,2008AJ....136..735L
7,150.456374000,28.785827000,I/317/sample,"PPMXL===4211247675463393029&-c=150.456374 +28.785827,eq=J2000&-c",--,0.0,138550,0.005,0.00015,2MASS:Ks,2010AJ....139.2440R
8,150.456374000,28.785827000,I/317/sample,"PPMXL===4211247675463393029&-c=150.456374 +28.785827,eq=J2000&-c",--,0.0,181750,0.00215,0.0001,2MASS:H,2010AJ....139.2440R


In [24]:
ned_t.show_in_notebook()

idx,No.,Observed_Passband,Photometry_Measurement,Uncertainty,Units,Frequency,Flux_Density,Upper_limit_of_uncertainty,Lower_limit_of_uncertainty,Upper_limit_of_Flux_Density,Lower_limit_of_Flux_Density,NED_Uncertainty,NED_Units,Refcode,Significance,Published_frequency,Frequency_Mode,Coordinates_Targeted,Spatial_Mode,Qualifiers,Comments
,,,,,,Hz,Jy,,,,,,,,,,,,,,
0,1,14-195 keV (Swift),9e-12,,erg/cm^2^/s,2.53e+19,3.56e-08,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,104.50 keV,Broad-band measurement,,Modelled datum,Intrinsic flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
1,2,14-195 keV (Swift),7.5e-12,,erg/cm^2^/s,2.53e+19,2.96e-08,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,104.50 keV,Broad-band measurement,,Modelled datum,Observed flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
2,3,14-195 keV (Swift),5.82e-12,+/-4.00E-12,erg/s/cm^2^,2.53e+19,2.3e-08,1.58e-08,1.58e-08,--,--,+/-1.58E-08,Jy,2018ApJS..235....4O,90% confidence,104.50 keV,Broad-band measurement,150.50804460 +28.79798862 (J2000),From fitting to map,S/N = 4.18 ; power law model,From new raw data; NED frequency assigned to mid-point ofband in keV
3,4,15-150 keV (Swift),5.6e-12,+/-3.5e-12,erg/cm^2^/s,1.99e+19,2.81e-08,1.76e-08,1.76e-08,--,--,+/-1.76E-08,Jy,2010A&A...524A..64C,uncertainty,82.50 keV,Broad-band measurement,150.458 +28.789 (J2000),Modelled datum,,From new raw data; NED frequency assigned to mid-point ofband in keV
4,5,14-150 keV (Swift),7.9e-12,,erg/cm^2^/s,1.98e+19,3.99e-08,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,82.00 keV,Broad-band measurement,,Modelled datum,Intrinsic flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
5,6,20-50 keV (Swift),4.3e-12,,erg/cm^2^/s,8.46e+18,5.08e-08,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,35.00 keV,Broad-band measurement,,Modelled datum,Intrinsic flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
6,7,2-10 keV (XMM),1.318e-12,,erg/cm^2^/s,1.45e+18,9.09e-08,--,--,--,--,,Jy,2008A&A...480..671P,no uncertainty reported,6.00 keV,Broad-band measurement,10 01 49.8 +28 47 08 (J2000),Modelled datum,Model B; see text,From new raw data; NED frequency assigned to mid-point ofband in keV
7,8,2-10 keV (XMM),1.327e-12,,erg/cm^2^/s,1.45e+18,9.15e-08,--,--,--,--,,Jy,2008A&A...480..671P,no uncertainty reported,6.00 keV,Broad-band measurement,10 01 49.8 +28 47 08 (J2000),Modelled datum,Model A; see text,From new raw data; NED frequency assigned to mid-point ofband in keV
8,9,2-10 keV (ASCA),1.17e-12,,ergs/s/cm^2^,1.45e+18,8.07e-08,--,--,--,--,,Jy,2001ApJS..133....1U,no uncertainty reported,6.00 keV,Broad-band measurement,150.4661 +28.7837 (J2000),Flux integrated from map,,From reprocessed raw data; Extinction-corrected for MilkyWay; NED frequency assigned to mid-point of band in keV


In [32]:
Ne_IR_Fq = np.array([19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0])*u.Hz
Ne_IR_Fq_GHz = Ne_IR_Fq.to(u.GHz)
Ne_IR_Fq_GHz

<Quantity [19200., 19300., 20900., 21000., 23400.] GHz>

In [33]:
for i, id_ in enumerate(NeIR_obj_t['Main_Id']):
    cds_table = Table.read(path_phot_cds+NeIR_obj_t['Main_Id'][i]+'_CDS_phot_tables'+'.vot',
                           format='votable')
    ned_table = Table.read(path_phot_ned+NeIR_obj_t['Main_Id'][i]+'_NED_phot_tables'+'.vot',
                           format='votable')
    Fq = np.array([ned_table['Frequency']])*u.Hz
    Fr = np.array([cds_table['sed_freq']])*u.GHz
    ints_cds = np.intersect1d(Ne_IR_Fq_GHz, Fr)
    ints_ned = np.intersect1d(Ne_IR_Fq, Fq)
    #print(ints_cds)
    if len(ints_cds)>0:
        print(i,' YES CDS')
    #if len(ints_ned)>0:
    #    print(i,' YES NED')
        

In [34]:
Ne_info = ['NeII|NeIII|NeV','NeII|NeV','NeIII|NeV']
ind_sample = [np.where(NeIR_obj_t['NED_Ne_IR_info'] == Ne_inf)[0] for Ne_inf in Ne_info]
ind_sample

[array([  0,   3,   4,   5,   6,   8,  11,  18,  20,  23,  24,  25,  27,
         28,  29,  30,  33,  34,  36,  38,  40,  41,  42,  43,  44,  46,
         47,  48,  49,  55,  57,  58,  60,  62,  63,  64,  65,  67,  69,
         70,  71,  77,  78,  81,  86,  88,  90,  91,  94,  96,  97, 102,
        105, 107, 108, 109, 110, 111, 112, 113, 114, 116, 117, 120, 125,
        126, 127, 131, 134, 139, 141, 143, 144, 148, 152, 153, 154, 155,
        156, 157, 159, 160, 161, 164, 167, 169, 170, 171, 177, 179, 180,
        181, 183, 190, 192, 193, 195, 196, 197, 198, 199, 200, 201, 204,
        207, 209, 214, 215, 218, 221, 226, 228, 232, 234, 236, 237, 239,
        241, 243, 244, 245, 247, 248, 249, 251, 253, 254, 257, 258, 259,
        264, 265, 267, 268, 271, 276, 277, 278, 280, 282, 287, 288, 289,
        290, 296, 297, 298, 299, 301, 303, 304, 305, 306, 307, 308, 311,
        313, 317, 320, 321, 323, 328, 329, 330, 333, 334, 336, 338, 339,
        342, 345, 346, 349, 350, 352, 353, 354]),
 

In [42]:
flx_ned = str(ned_table[3]['Flux_Density'])
flx_ned

'1.34e-07'

In [35]:
cds_table[377]

_RAJ2000,_DEJ2000,_tabname,_ID,_tab1_29,_tab1_30,sed_freq,sed_flux,sed_eflux,sed_filter,Bibcode
deg,deg,,,d,s,GHz,Jy,Jy,---,
float64,float64,str32,str64,float64,float64,float64,float32,float32,str32,object
120.087500000,26.613611000,J/MNRAS/456/3335/table12,recno=106,--,0.0,599.59,0.12,--,:=500um,2016MNRAS.456.3335S


In [17]:
for tabindx, tabinfo in enumerate(cds_tablee['_tabname']):
    try:
        time.sleep(0.001)  # This is used to avoid being flag by the server
        name_cat = tabinfo.rpartition('/')[0]
        search = Vizier.query_constraints(catalog='METAcat', name=name_cat)
        cds_tablee['Bibcode'][tabindx] = search[0][0]['bibcode']
    except IndexError:
        print('There is an error at ', tabindx, tabinfo)

In [18]:
cds_tablee

_RAJ2000,_DEJ2000,_tabname,_ID,_tab1_31,_tab1_32,sed_freq,sed_flux,sed_eflux,sed_filter,Bibcode
deg,deg,,,d,s,GHz,Jy,Jy,,
float64,float64,str32,str64,float64,float64,float64,float32,float32,str32,object
150.456380600,28.785966700,I/297/out,"NOMAD1===1187-0182426&-c=150.4563806 +28.7859667,eq=J2000&-c.rs=",--,0.0,138550,0.00501,--,2MASS:Ks,2004AAS...205.4815Z
150.456380600,28.785966700,I/297/out,"NOMAD1===1187-0182426&-c=150.4563806 +28.7859667,eq=J2000&-c.rs=",--,0.0,181750,0.00215,--,2MASS:H,2004AAS...205.4815Z
150.456380600,28.785966700,I/297/out,"NOMAD1===1187-0182426&-c=150.4563806 +28.7859667,eq=J2000&-c.rs=",--,0.0,241960,0.00116,--,2MASS:J,2004AAS...205.4815Z
150.456445000,28.785850000,I/305/out,"GSC2.3===N9AE005113&-c=150.456445 +28.785850,eq=J2000&-c.rs=0.00",--,0.0,382540,0.0021,0.00083,POSS-II:i,2008AJ....136..735L
150.456445000,28.785850000,I/305/out,"GSC2.3===N9AE005113&-c=150.456445 +28.785850,eq=J2000&-c.rs=0.00",--,0.0,468430,0.000992,0.00039,POSS-II:F,2008AJ....136..735L
150.456445000,28.785850000,I/305/out,"GSC2.3===N9AE005113&-c=150.456445 +28.785850,eq=J2000&-c.rs=0.00",--,0.0,640580,0.000753,0.00031,POSS-II:J,2008AJ....136..735L
150.456445000,28.785850000,I/305/out,"GSC2.3===N9AE005113&-c=150.456445 +28.785850,eq=J2000&-c.rs=0.00",--,0.0,674900,0.000364,0.00012,Johnson:B,2008AJ....136..735L
150.456374000,28.785827000,I/317/sample,"PPMXL===4211247675463393029&-c=150.456374 +28.785827,eq=J2000&-c",--,0.0,138550,0.005,0.00015,2MASS:Ks,2010AJ....139.2440R


In [32]:
name_cat = tabinfo.rpartition('/')[0]
name_cat

'J/other/APh/26.282'

In [27]:
search = Vizier.query_constraints(catalog='METAcat', name=name_cat)

In [31]:
search[search.keys()[0]].pprint()

catid kslot catype        name        ...       bibcode       popu authid
----- ----- ------ ------------------ ... ------------------- ---- ------
  255    63      5 J/other/APh/26.282 ... 2006APh....26..282A  255      0


In [29]:
search[0][0]['bibcode']

'2006APh....26..282A'

In [39]:
c = [(a + 2) for a in range(10)] 
c

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]